In [ ]:
# Coding GenAi RAG set question 

In [5]:
GEMINI_API_KEY="key"
OPENAI_API_KEY = "key"

Question 1:

Task: Implement a FAISS-based Retrieval System that fetches the most relevant document for query-based retrieval.


In [4]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 1. Documents
documents = [
    "RAG combines retrieval and generation.",
    "FAISS is a vector search library.",
    "Generative AI creates new content."
]

# 2. Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 3. Create embeddings
doc_embeddings = model.encode(documents)

# 4. Create FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

# 5. Query
query = "What is RAG?"
query_embedding = model.encode([query])

# 6. Search
k = 1
distances, indices = index.search(np.array(query_embedding), k)

# 7. Result
print("Most relevant document:")
print(documents[indices[0][0]])


Most relevant document:
RAG combines retrieval and generation.


Question 2:

Task: Use Gemini API to generate answers based on retrieved knowledge.


In [6]:
from google import genai

# 1. Create Gemini client
client = genai.Client(api_key=GEMINI_API_KEY)

# 2. Retrieved document (from FAISS)
retrieved_doc = "RAG combines retrieval and generation."

# 3. User query
query = "very short explain about RAG."

# 4. Prompt with retrieved knowledge
prompt = f"""
Use the following information to answer the question.

Information:
{retrieved_doc}

Question:
{query}
"""

# 5. Generate answer
response = client.models.generate_content(
    model="models/gemini-2.5-flash",
    contents=prompt
)

# 6. Output
print(response.text)


RAG combines retrieval and generation.


Question 3:

Task: Implement FAISS with a hybrid (dense & sparse) vector search for improving information retrieval.


In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Documents
documents = [
    "RAG combines retrieval and generation.",
    "FAISS is a vector search library.",
    "Generative AI creates new content."
]

# 2. Dense embeddings (semantic)
dense_model = SentenceTransformer("all-MiniLM-L6-v2")
dense_embeddings = dense_model.encode(documents)

# 3. FAISS index (dense)
dim = dense_embeddings.shape[1]
dense_index = faiss.IndexFlatL2(dim)
dense_index.add(np.array(dense_embeddings))

# 4. Sparse embeddings (keywords)
tfidf = TfidfVectorizer()
sparse_embeddings = tfidf.fit_transform(documents).toarray()

# 5. Query
query = "What is RAG?"
query_dense = dense_model.encode([query])
query_sparse = tfidf.transform([query]).toarray()

# 6. Dense search
k = 3
dense_scores, dense_ids = dense_index.search(np.array(query_dense), k)

# 7. Sparse similarity (dot product)
sparse_scores = np.dot(sparse_embeddings, query_sparse.T).flatten()

# 8. Hybrid score (simple sum)
final_scores = {}
for i in range(len(documents)):
    dense_score = -dense_scores[0][list(dense_ids[0]).index(i)] if i in dense_ids[0] else 0
    sparse_score = sparse_scores[i]
    final_scores[i] = dense_score + sparse_score

# 9. Best document
best_doc = max(final_scores, key=final_scores.get)

print("Most relevant document:")
print(documents[best_doc])


Most relevant document:
RAG combines retrieval and generation.


Question 4:

Task: Implement a Multi-Turn Conversational Chatbot using Gemini API that maintains context across multiple user interactions.


In [9]:
from google import genai

# 1. Create Gemini client
client = genai.Client(api_key=GEMINI_API_KEY)


MODEL_NAME = "models/gemini-2.5-flash"

history = []

while True:
    user_input = input("User: ")

    if user_input.lower() == "exit":
        break

    # Add user message
    history.append(
        genai.types.Content(
            role="user",
            parts=[genai.types.Part(text=user_input)]
        )
    )

    # Generate response
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=history
    )

    reply = response.text
    print("Bot:", reply)

    # Add bot reply
    history.append(
        genai.types.Content(
            role="model",
            parts=[genai.types.Part(text=reply)]
        )
    )



User:  how are you?


Bot: As an AI, I don't have feelings or a physical body, so I can't be "fine" in the human sense. However, I am functioning perfectly and ready to help you!

How are you doing today? And what can I assist you with?


User:  what your name?


Bot: I do not have a name. I am a large language model, trained by Google.


User:  exit


In [ ]:
Question 5:

Task: Implement a FAISS-based retrieval-augmented generation (RAG) system using the Gemini API, where retrieved documents are used as context to generate more accurate and relevant responses.


In [10]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google import genai

# 1. Gemini client
client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_NAME = "models/gemini-2.5-flash"

# 2. Documents (knowledge base)
documents = [
    "RAG combines retrieval and generation to improve accuracy.",
    "FAISS is a fast vector similarity search library.",
    "Gemini is a large language model by Google."
]

# 3. Load embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# 4. Create document embeddings
doc_embeddings = embed_model.encode(documents)

# 5. Create FAISS index
dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(doc_embeddings))

# 6. User query
query = "Explain RAG in short form"

# 7. Embed query
query_embedding = embed_model.encode([query])

# 8. Retrieve top document
k = 1
_, indices = index.search(np.array(query_embedding), k)
retrieved_doc = documents[indices[0][0]]

# 9. Create RAG prompt
prompt = f"""
Use the following information to answer the question.

Context:
{retrieved_doc}

Question:
{query}
"""

# 10. Generate answer using Gemini
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=prompt
)

# 11. Output
print("Answer:")
print(response.text)


Answer:
RAG combines retrieval and generation to improve accuracy.
